<a href="https://colab.research.google.com/github/budvinchathura/kvasir/blob/master/cnn_tf_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# ! wget 'https://datasets.simula.no/kvasir/data/kvasir-dataset-v2.zip'
# ! unzip 'kvasir-dataset-v2.zip'

In [0]:
import cv2
import numpy as np
import os

from random import shuffle
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
import tflearn

from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression

from sklearn.model_selection import KFold

%matplotlib inline

In [0]:
# IMG_SIZE = 150
# LR = 1e-3
# MODEL_NAME='cats-and-dogs'

In [0]:
IMG_SIZE = 200
LR = 1e-3
MODEL_NAME='kvasir'

In [0]:
# def create_data(path):
#   data = []
#   for img_name in tqdm(os.listdir(path+'/cats')):
#     img_data = cv2.imread(path+'/cats/'+img_name,cv2.IMREAD_GRAYSCALE)
#     img_data = cv2.resize(img_data,(IMG_SIZE,IMG_SIZE))
#     data.append([np.array(img_data),np.array([1,0])])

#   for img_name in tqdm(os.listdir(path+'/dogs')):
#     img_data = cv2.imread(path+'/dogs/'+img_name,cv2.IMREAD_GRAYSCALE)
#     img_data = cv2.resize(img_data,(IMG_SIZE,IMG_SIZE))
#     data.append([np.array(img_data),np.array([0,1])])
#   shuffle(data)
#   return data

In [0]:
def create_data(path,label_n):
  label=[0]*8
  label[label_n] = 1
  data = []
  for img_name in tqdm(os.listdir(path)):
    img_data = cv2.imread(path+'/'+img_name,cv2.IMREAD_GRAYSCALE)
    img_data = cv2.resize(img_data,(IMG_SIZE,IMG_SIZE))
    data.append([np.array(img_data),np.array(label)])

  
  return data

In [0]:
# training_data = create_data('cats_and_dogs_filtered/train')
# np.save('train_data.npy',training_data)
# test_data = create_data('cats_and_dogs_filtered/validation')
# np.save('test_data.npy',test_data)

In [23]:
all_data = []
path='kvasir-dataset-v2'
i=0
for img_class in tqdm(os.listdir(path)):
  all_data.extend(create_data(path+'/'+img_class,i))
  i+=1
shuffle(all_data)


100%|██████████| 8/8 [01:10<00:00,  9.95s/it]


In [0]:
all_data = np.array(all_data)

images_n = all_data.shape[0]

training_data=all_data[:]
# test_data=all_data[images_n//2:]

# kf = KFold(5,shuffle=True)


In [0]:
x_train = np.array([i[0] for i in training_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_train = [i[1] for i in training_data]

# x_test = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
# y_test = [i[1] for i in test_data]

# print(x_train[3])

In [0]:
tf.reset_default_graph()
convnet = input_data(shape=[None,IMG_SIZE,IMG_SIZE,1],name='input')   #feeding input data

convnet = conv_2d(convnet,16,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,16,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,16,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)


convnet = conv_2d(convnet,32,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,32,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,32,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,32,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,32,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,64,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,64,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)


convnet = conv_2d(convnet,64,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,64,3,strides=1,activation='relu')
convnet = max_pool_2d(convnet,2)








convnet = fully_connected(convnet,8,activation='softmax')

convnet = regression(convnet,optimizer='adam',learning_rate=LR,loss='categorical_crossentropy',name='targets')

model = tflearn.DNN(convnet,tensorboard_dir='log',tensorboard_verbose=0)





In [27]:
model.fit({'input':x_train},{'targets':y_train},
          n_epoch=100,validation_set=0.25,
          snapshot_step=500,show_metric=True,run_id=MODEL_NAME)


# from sklearn import metrics





# for passes in range(2):
#   for k, (train_indices, test_indices) in enumerate(kf.split(training_data)):

#     x_train = np.array([i[0] for i in training_data[train_indices]]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
#     y_train = [i[1] for i in training_data[train_indices]]
#     # print(x_train[0])
#     # print(y_train[0])


#     x_test = np.array([i[0] for i in training_data[test_indices]]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
#     y_test = [i[1] for i in training_data[test_indices]]



#     model.fit({'input':x_train},{'targets':y_train},
#             n_epoch=10,validation_set=({'input':x_test},{'targets':y_test}),
#             snapshot_step=500,show_metric=True,run_id=MODEL_NAME)
    # y_pred = model.predict(x_test)
    # print(y_pred)
    # results = model.evaluate({'input':x_test},{'targets':y_test})
    # print(metrics.accuracy_score(y_test, y_pred))

    
    

Training Step: 9399  | total loss: 0.03319 | time: 6.418s
| Adam | epoch: 100 | loss: 0.03319 - acc: 0.9907 -- iter: 5952/6000
Training Step: 9400  | total loss: 0.03626 | time: 7.491s
| Adam | epoch: 100 | loss: 0.03626 - acc: 0.9885 | val_loss: 2.43624 - val_acc: 0.6395 -- iter: 6000/6000
--
